In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 3, Finished, Available, Finished)

In [ ]:
df = spark.sql("SELECT * FROM Lakehouse_Demo.dim_customer LIMIT 1000")
display(df)

In [4]:
%%sql
create table if not EXISTS Lakehouse_Demo.fact_sales
(
Order_ID	string	,
Price double,
Quantity	Int	,
Sales	double	,
Discount	double	,
Profit	double	,
Shipping_Cost	double	,
Order_Date	date	,
Shipping_Date	date	,
Product_ID	long	,
OrderPriority_ID	Long	,
ShipMode_ID	Long	,
Customer_ID	string	,
Order_Year	integer	,
Order_Month	integer	,
Created_TS	timestamp	,
Modified_TS	timestamp	
)
using DELTA
PARTITIONED by (Order_Year,Order_Month)

StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
Max_Date = spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Lakehouse_Demo.fact_sales").first()[0]

print(Max_Date)

StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 7, Finished, Available, Finished)

1900-01-01


In [6]:
df_sales = spark.sql("""
SELECT 
    s.Order_ID,
    s.Sales AS Price,
    s.Quantity,
    s.Sales * s.Quantity AS Sales,
    s.Discount,
    s.Profit,
    s.Shipping_Cost,
    s.Order_Date,
    s.Shipping_Date,
    p.Product_ID,
    op.OrderPriority_ID,
    sm.ShipMode_ID,
    s.Customer_ID,
    YEAR(s.Order_Date) AS Order_Year,
    MONTH(s.Order_Date) AS Order_Month
FROM Lakehouse_Demo.Sales_Stg s
INNER JOIN Lakehouse_Demo.Dim_Product p 
    ON s.Product = p.Product 
   AND s.Product_Category = p.Product_Category
INNER JOIN Lakehouse_Demo.Dim_shipmode sm
    ON s.Ship_Mode = sm.Ship_Mode
INNER JOIN Lakehouse_Demo.Dim_OrderPriority op
    ON s.Order_Priority = op.Order_Priority
""")


StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 8, Finished, Available, Finished)

In [7]:
df_sales.createOrReplaceTempView("ViewFactSale")

StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 9, Finished, Available, Finished)

In [8]:
%%sql
Merge into Lakehouse_Demo.fact_sales as fs
using ViewFactSale as vfs 
on fs.Order_Year=vfs.Order_Year and fs.Order_Month=vfs.Order_Month and fs.Order_ID=vfs.Order_ID
when matched then 
update SET
fs.Sales	=	vfs.Sales	,
fs.Price	=	vfs.Price	,
fs.Quantity	=	vfs.Quantity	,
fs.Discount	=	vfs.Discount	,
fs.Profit	=	vfs.Profit	,
fs.Shipping_Cost	=	vfs.Shipping_Cost	,
fs.Order_Date	=	vfs.Order_Date	,
fs.Shipping_Date	=	vfs.Shipping_Date	,
fs.Product_ID	=	vfs.Product_ID	,
fs.OrderPriority_ID	=	vfs.OrderPriority_ID	,
fs.ShipMode_ID	=	vfs.ShipMode_ID	,
fs.Customer_ID	=	vfs.Customer_ID	,
fs.Modified_TS	=	current_timestamp()	

when not matched then
INSERT
(
Order_ID	,
Sales	,
Price,
Quantity	,
Discount	,
Profit	,
Shipping_Cost	,
Order_Date	,
Shipping_Date	,
Product_ID	,
OrderPriority_ID	,
ShipMode_ID	,
Customer_ID	,
Order_Year	,
Order_Month	,
Created_TS	,
Modified_TS	

)
VALUES
(
vfs.Order_ID	,
vfs.Sales	,
vfs.Price,
vfs.Quantity	,
vfs.Discount	,
vfs.Profit	,
vfs.Shipping_Cost	,
vfs.Order_Date	,
vfs.Shipping_Date	,
vfs.Product_ID	,
vfs.OrderPriority_ID	,
vfs.ShipMode_ID	,
vfs.Customer_ID	,
vfs.Order_Year	,
vfs.Order_Month	,
current_timestamp()	,
current_timestamp()

)

StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [9]:
%%sql
select * from Lakehouse_Demo.fact_sales

StatementMeta(, 3bc37a56-05c1-4057-842d-338ce46e7282, 11, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 17 fields>